In [ ]:
# Bismillah

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
# from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('precision', 7)
from pandas_datareader import data as web

In [ ]:
msft = web.DataReader('MSFT', 'yahoo', '2012-01-01', '2013-12-30')
msft

# collects the data from yahoo finance for Microsoft for the said period and stores it in data frame, msft.

### Shifting and lagging time-series data:

In [ ]:
msftAC = msft['Adj Close']
msftAC

# select only 'Adj Close' column from the msft data frame, and store it in a series, msftAC.

In [ ]:
type(msftAC)

# msftAC is a series. We can convert it to a data frame by typing: msftAC = msft[['Adj Close']]

In [ ]:
shift_forward = msftAC.shift(1)
msftAC.head(), shift_forward.head()
# This will shift the prices one day forward.
# results in one NaN value at the start since there was no value prior to be shifted to 2012-01-03.
# the number of NaN values will depend on the number of indices we are shifting forward or backward. For instance, the index is 
# daily dates, so if we shift one day forward, it means we are forwarding one index resulting into one NaN value. If we would 
# shift 2 days forward, it means we are forwarding two indices resulting into two NaN values.

In [ ]:
msftAC.tail(), shift_forward.tail()
# the last price 31.9755402 on date 2013-12-30 in msftAC has been lost away in shift_forward since there is no index available for
# it now after the prices have been shifted one day forward. So, we got one NaN value at the start and one value lost at the end
# by shifting one day forward.

In [ ]:
shift_backward = msftAC.shift(-2)
# will shift two days (two indices) backward.

In [ ]:
msftAC, shift_backward
# inserts two NaNs at the end since we have shifted two days (two indices) backward and the first two values lost since there are 
# no indices available for them.

In [ ]:
shift_1sec = msftAC.shift(1, freq = 'S')

In [ ]:
msftAC, shift_1sec
# the prices have been shifted just one second forward each day. That means that the prices still exist on the same date. So, no
# value is lost and no NaN is inserted.
# the frequency could be changed to any desirable level such as hour etc.
# Note that the index labels have been adjusted according to the specified frequency resulting into a new dataframe or series in 
# this case.

In [ ]:
shift_1sec['2012-01-03':'2012-01-06']
# note that the indices now include the time as well in addition to the date.

### Practical Example:

In [ ]:
msftAC, msftAC.shift(1)
# the prices have been shifted one day forward.

In [ ]:
msftAC/msftAC

In [ ]:
msftAC / msftAC.shift(1)
# the dividsion is based on the same index labels. That is, the price on 2012-01-04 in msftAC is divided the price on 2012-01-04 in
# msftAC.shift(1). This is equal t`o (current price divided by previous price). The previous price is the amount invested.

# Interpretation: the $1 invested has grown to $1.0235337. So, the net return is $1.0235337 - 1 = $0.0235337. That is why we need
# to subtract 1 as shown below.
# the first value is NaN because value is missing in msftAC.shift(1) on 2012-01-03.
# the last value is 1.00000 because in both msftAC and msftAC.shift(1) the value on 2013-12-30 is the same.

# Very important to note that we CANNOT say that $1 investment at the start (2012-01-03) has grown to (or in this case 
# has remained same) to $1 at the end of the period (2013-12-30) since we can see that the adj close price at the start
# is $21.62 and at the end is $31.90. So, there is definately a gain. But then why we have 1 at the end of the output of
# this cell? It is because these returns are computed on daily basis. So, this 1 shows that $1 dollar invested on 2013-12
#-27 has remained same $1 and it makes sense since the price has remained unchanged from 2013-12-27 to 2013-12-30. To find
# the total return, we need to find the cumulative product.

In [ ]:
np.cumprod(msftAC / msftAC.shift(1))

# just for the confirmation of the above.

In [ ]:
31.9095650 / 21.6231346

# just for the confirmation above

In [ ]:
percent_change = msftAC / msftAC.shift(1) - 1
percent_change
# gives the net daily change or return.
# the price has increased by 2.3% from 2012-01-03 to 2012-01-04.

### Frequency Conversion of Time-Series Data:

In [ ]:
sample = msftAC[:2]
sample
# selected first 2 rows and stored them in 'sample'

In [ ]:
sample.asfreq('H')
# to have hourly data between the two index labels.
# a new index with hourly index labels has been created by pandas, but when aligning to the original time series, only two values
# were found, thereby leaving the others filled with NaN. This default behaviour can be changed as follows:

In [ ]:
sample.asfreq('H', method = 'ffill')
# forward fill will take the earlier known value and fill all the missing values with it.

In [ ]:
sample.asfreq('H', method = 'bfill')
# backward fill will take the next known value and fill the missing values in the backward direction.

### Resampling of time-series:

In [ ]:
msftAC / msftAC.shift()
# I haven't confirmed it but i think the default argument for shift is 1 which means to shift prices forward by one day.

In [ ]:
(1 + (msftAC / msftAC.shift() - 1))
# this and immediately above seems to produce the same result. Plus1 and minus1 cancels out so the result is the same as immediately above.

In [ ]:
msft_cum_ret = (1 + (msftAC / msftAC.shift() - 1)).cumprod()
msft_cum_ret
# cumprod stands for cummulative product.
# it multiplies the value on 2012-01-04 with the value on 2012-01-05 and produces the product in the index label 2012-01-05. This product 
# can be called cumulative daily return.
# $1 dollar invested on 2012-01-03 has grown to $1.475 on 2013-12-30.

In [ ]:
msft_cum_ret['1/1/2012':'1/31/2012']
# note that typing the date in this pattern is the same as typing 2012-01-04 and so on.
# this gives us daily cumulative returns for all the month as explained above.

In [ ]:
msft_cum_ret['1/1/2012':'1/31/2012'].mean()
# mean cumulative return of January 2012. It is a mean of all the daily cumulative returns in January.
# to calculate each month mean cumulative return, we have to write this command 24 times for all 24 months.

In [ ]:
msft_cum_ret['1/2012'].mean()
# an alternative to the above method to compute monthly mean.

In [ ]:
(msftAC/msftAC.shift()).mean() # mean of daily returns.

In [ ]:
msft_cum_ret

In [ ]:
msft_monthly_cum_ret = msft_cum_ret.resample("M")
msft_monthly_cum_ret
# a better and efficient alternative to the above two methods to compute monthly means.
# does not return a dataframe any longer but an object stored at some location in memory. Nevermind, all the aggregation and interpolation
# can still be validly performed as shown below.

In [ ]:
def print_info(groupobject):
    for i, k in groupobject:
        print (i)
        print (k)

In [ ]:
print_info(msft_monthly_cum_ret)
# the daily cumulative returns have been grouped by month. In total, we will have 24 groups since we have 24 months.

In [ ]:
len(msft_monthly_cum_ret)
# 24 groups confirmed.

In [ ]:
msft_monthly_cum_ret.size()
# size of each group.

In [ ]:
msft_monthly_cum_ret.mean()

In [ ]:
msft_cum_ret['2/2012'].mean()
# to compare and confirm the mean of feb in immediately above series. Confirmed and the same.

In [ ]:
## The above work is almost the end of page 94 in the book. After that, for about 3 pages, the discussion in the book cannot be
# implemented here since there is change in the newer version of pandas. But what we can do something extra and interesting follows
# as:

In [ ]:
msft

In [ ]:
msft_resample = msft.resample('M')
# the frequency has been changed to Monthly from Daily.

In [ ]:
msft_resample

In [ ]:
for i, k in msft_resample:
    print(i)
    print(k)
# the data has been grouped on monthly basis into 24 groups since we have 24 months.    

In [ ]:
msft_resample.mean().head(3)
# mean of high, low, open, close, volume and adj close for each month.

In [ ]:
msft.loc['1/2012', 'Adj Close'].mean()
# confirmed that 'msft_resample.mean()' has produced mean of high, low, open, close, volume and adj close for each month.